### 1.隐语义模型和矩阵分解

协同过滤算法的特点就是完全没有利用到物品本身或者是用户自身的属性， 仅仅利用了用户与物品的交互信息就可以实现推荐，是一个可解释性很强， 非常直观的模型， 但也存在缺陷：一是处理稀疏矩阵的能力比较弱， 为了使得协同过滤更好处理稀疏矩阵问题， 增强泛化能力， 从协同过滤中衍生出**矩阵分解模型（Matrix Factorization，MF)**或者叫**隐语义模型（latent factor model，LFM）**, 是在协同过滤共现矩阵的基础上， 使用更稠密的隐向量表示用户和物品，挖掘用户和物品的隐含兴趣和隐含特征， 在一定程度上弥补协同过滤模型处理稀疏矩阵能力不足的问题。

隐语义模型的核心思想是**通过隐含特征（latent  factor）联系用户兴趣和物品（item）， 基于用户的行为找出潜在的主题和分类， 然后对item进行自动聚类，划分到不同类别/主题(用户的兴趣)**。

通过分解用户-物品的共现矩阵来得到用户矩阵$Q$和物品矩阵$P$，进而得到用户和物品的隐向量。矩阵分解算法将$m\times n$维的共享矩阵$R$分解成$m \times k$维的用户矩阵$U$和$k \times n$维的物品矩阵$V$相乘的形式。 其中$m$是用户数量， $n$是物品数量， $k$是隐向量维度， 也就是隐含特征个数，$k$的大小决定了隐向量表达能力的强弱， $k$越大， 表达信息就越强， 理解起来就是把用户的兴趣和物品的分类划分的越具体。

> $k$的取值与矩阵分解的求解复杂度直接相关，在具体应用中$k$的取值要经过多次试验找到一个推荐效果和工程开销的平衡点。

#### 矩阵分解算法的求解

矩阵分解最常用的方法是特征值分解(EVD)或者奇异值分解(SVD)，这两种方法的推导过程在《矩阵理论》课程中都有讲到，但是这两种方式在这里不适用：

* 特征值分解要求矩阵是方阵， 显然用户-物品矩阵不满足这个要求；
* 传统的SVD分解要求原始矩阵是稠密的， 而实际应用中的共现矩阵是非常稀疏的， 若想用奇异值分解， 就必须对缺失的元素进行填充；传统SVD分解计算复杂度非常高，达$O(mn^2)$级别，对用户数量成千上万的互联网场景是不可接受的。

因此梯度下降法成了进行矩阵分解的主要方法，把求解上面两个矩阵的参数问题转换成一个最优化问题， 可以通过训练集里面的观察值利用最小化来学习用户矩阵和物品矩阵。

1. 设已有用户矩阵和物品矩阵， 估计用户$u$对物品$i$的评分：

$$
\operatorname{Preference}(u, i)=\hat{r}_{u i}=p_{u}^{T} q_{i}=\sum_{f=1}^{F} p_{u, k} q_{k,i}
$$

2. 预测值与真实值之间的误差：

$$
e_{u i}=r_{u i}-\hat{r}_{u i}
$$

3. 计算出所有样本的误差平方和SSE：

$$
\operatorname{SSE}=\sum_{u, i} e_{u i}^{2}=\sum_{u, i}\left(r_{u i}-\sum_{k=1}^{K} p_{u,k} q_{k, i}\right)^{2}
$$

4. 构造损失函数，为了减少过拟合现象加入正则化项：

$$
\min\limits_{\mathbf{q*},\mathbf{p*}} \sum\limits_{(u,i)\in K}(\mathbf{r}_{ui}-\mathbf{q}_i^T\mathbf{p}_u)^2+\lambda(||\mathbf{q}_i||^2+||\mathbf{p}_u||^2)
$$

5. 梯度下降，更新参数$\mathbf{q}_i$和$\mathbf{p}_u$；要注意的是：初始化用户矩阵$Q$和物品矩阵$P$时，根据经验随机数需要和1/sqrt(k)成正比。

**为了消除用户和物品打分的偏差（Bias)，常用的做法是在矩阵分解时加入用户和物品的偏差向量**：
$$
\hat{r}_{u i}=\mu+b_{u}+b_{i}+p_{u}^{T} \cdot q_{i}
$$
这个预测公式加入了3项偏置$\mu,b_u,b_i$, 作用如下：

- $\mu$: **训练集中所有记录的评分的全局平均数**。 在不同网站中， 因为网站定位和销售物品不同， 网站的整体评分分布也会显示差异。 比如有的网站中用户就喜欢打高分， 有的网站中用户就喜欢打低分。 而全局平均数可以表示网站本身对用户评分的影响。
- $b_u$: **用户偏差系数， 可以使用用户$u$给出的所有评分的均值， 也可以当做训练参数**。 这一项表示了用户的评分习惯中和物品没有关系的那种因素。 比如有些用户比较苛刻， 对什么东西要求很高， 那么他评分就会偏低， 而有些用户比较宽容， 对什么东西都觉得不错， 那么评分就偏高
- $b_i$: **物品偏差系数， 可以使用物品$i$收到的所有评分的均值， 也可以当做训练参数**。 这一项表示了物品接受的评分中和用户没有关系的因素。 比如有些物品本身质量就很高， 因此获得的评分相对比较高， 有的物品本身质量很差， 因此获得的评分相对较低。

代码实现如下：

In [6]:
import random
import math

In [7]:
class SVD():
    def __init__(self, rating_data, F=5, alpha=0.1, lmbda=0.1, max_iter=100):
        self.F = F           # 这个表示隐向量的维度
        self.P = dict()          #  用户矩阵P  大小是[users_num, F]
        self.Q = dict()     # 物品矩阵Q  大小是[item_nums, F]
        self.bu = dict()   # 用户偏差系数
        self.bi = dict()    # 物品偏差系数
        self.mu = 0.0        # 全局偏差系数
        self.alpha = alpha   # 学习率
        self.lmbda = lmbda    # 正则项系数
        self.max_iter = max_iter    # 最大迭代次数
        self.rating_data = rating_data # 评分矩阵
        
        # 初始化矩阵P和Q, 方法很多， 一般用随机数填充， 但随机数大小有讲究， 根据经验， 随机数需要和1/sqrt(F)成正比
        cnt = 0    # 统计总的打分数， 初始化mu用
        for user, items in self.rating_data.items():
            self.P[user] = [random.random() / math.sqrt(self.F)  for x in range(0, F)]
            self.bu[user] = 0
            cnt += len(items) 
            for item, rating in items.items():
                if item not in self.Q:
                    self.Q[item] = [random.random() / math.sqrt(self.F) for x in range(0, F)]
                    self.bi[item] = 0
        self.mu /= cnt
        
    # 有了矩阵之后， 就可以进行训练, 这里使用随机梯度下降的方式训练参数P和Q
    def train(self):
        for step in range(self.max_iter):
            for user, items in self.rating_data.items():
                for item, rui in items.items():
                    rhat_ui = self.predict(user, item)   # 得到预测评分
                    # 计算误差
                    e_ui = rui - rhat_ui
                    
                    self.bu[user] += self.alpha * (e_ui - self.lmbda * self.bu[user])
                    self.bi[item] += self.alpha * (e_ui - self.lmbda * self.bi[item])
                    # 随机梯度下降更新梯度
                    for k in range(0, self.F):
                        self.P[user][k] += self.alpha * (e_ui*self.Q[item][k] - self.lmbda * self.P[user][k])
                        self.Q[item][k] += self.alpha * (e_ui*self.P[user][k] - self.lmbda * self.Q[item][k])
                    
            self.alpha *= 0.1    # 每次迭代步长要逐步缩小
    
    # 预测user对item的评分， 这里没有使用向量的形式
    def predict(self, user, item):
        return sum(self.P[user][f] * self.Q[item][f] for f in range(0, self.F)) + self.bu[user] + self.bi[item] + self.mu   

In [8]:
# 定义数据集， 也就是那个表格， 注意这里我们采用字典存放数据， 因为实际情况中数据是非常稀疏的， 很少有情况是现在这样
def loadData():
    rating_data={1: {'A': 5, 'B': 3, 'C': 4, 'D': 4},
           2: {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3},
           3: {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5},
           4: {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4},
           5: {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}
          }
    return rating_data
 
# 接下来就是训练和预测
rating_data = loadData()
basicsvd = SVD(rating_data, F=10)
basicsvd.train()
for item in ['E']:
    print(item, basicsvd.predict(1, item))

E 3.2318929368219655


**相比于人工指定的物品分类，隐语义模型较好地解决了几个问题**：

1. 隐语义技术的分类来自对用户行为的统计，代表了用户对物品的看法；
2. 允许指定最终有多少个分类，数字越大，分类的粒度就越细；
3. 会计算出物品属于每个类的权重，因此每个物品不是硬性地被分到某一个类；
4. 每个分类都不是同一维度的，它是基于用户的共同兴趣计算出来的；
5. 可以通过统计用户行为决定物品在每个类中的权重。

**相比于协同过滤，矩阵分解有如下优点**：

1. 泛化能力强，一定程度上解决了数据稀疏问题；
2. 空间复杂度低，$O((n+m)k)$级别；
3. 更好的扩展性和灵活性，便于与其他特征进行组合和拼接，并便于与深度学习网络进行无缝结合（其思想与Embedding不谋而合）。

当然**矩阵分解模型也有局限性**：它同样不方便加入用户、物品和上下文相关的特征，丧失了利用很多有效信息的机会，同时在缺乏用户历史行为时，无法进行有效的排序。